In [2]:
import pandas as pd
import pickle
import rqdatac as rqd
rqd.init()

In [119]:
def get_statistical_indicators(path):
    with open(path,'rb') as f:
        config,result = pickle.load(f)
    result = result['sys_analyser']
    summary = {}
    # ========相关数据========
    # 起始日
    summary['起始日'] = result['summary']['start_date']
    # 标的资产
    summary['标的资产'] = config['mod']['sys_analyser']['benchmark']
    # 股票数
    latest_date = result['stock_positions'].index.unique()[-1]
    positions = result['stock_positions'].loc[latest_date]
    positions.set_index('order_book_id',inplace=True)
    positions['weight'] = positions.loc[:,'market_value']/positions.loc[:,'market_value'].sum()
    summary['股票数'] = positions.shape[0]
    # Beta
    summary['Beta'] = result['summary']['beta']
    # 跟踪误差
    summary['跟踪误差'] = result['summary']['tracking_error']
    # 年化收益率
    summary['年化收益率'] = result['summary']['annualized_returns']
    # 基准年化收益率
    summary['基准年化收益率'] = result['summary']['benchmark_annualized_returns']
    # 十大持仓股
    summary['十大持仓股'] = positions.nlargest(10,'weight')[['symbol','weight']]
    # 行业配置
    positions['industry'] = rqd.get_instrument_industry(positions.index,source='sws',date=latest_date)['first_industry_name']
    summary['行业配置'] = positions.groupby('industry')['weight'].sum().sort_values(ascending=False).to_frame()
    # 持仓净值
    net_value = result['benchmark_portfolio'].rename(columns={'unit_net_value':'基准'})
    net_value['单位净值'] = result['portfolio']['unit_net_value']
    net_value = net_value[['单位净值','基准']]
    summary['净值'] = net_value
    # 近一月收益率
    summary['近一个月收益率'] = net_value.iloc[-1,0]/net_value.iloc[-21,0]-1
    # 组合与基准收益
    profit_df = pd.DataFrame(index=['组合资产','标的资产'],columns=['1年','3年','5年','10年','至今']) 
    profit_df['至今'] = (net_value.iloc[-1,:]/net_value.iloc[0,:]-1).values
    profit_df['1年'] = (net_value.iloc[-1,:]/net_value.iloc[-252*1,:]-1).values
    profit_df['3年'] = (net_value.iloc[-1,:]/net_value.iloc[-252*3,:]-1).values
    profit_df['5年'] = (net_value.iloc[-1,:]/net_value.iloc[-252*5,:]-1).values
    summary['组合与基准收益'] = profit_df
    
    return summary

In [120]:
path = 'D:/jpnb/ricequant/ricequant实习项目/ricequant/AMS/基金重仓股策略/result/基金重仓股_300_规模大于5亿.pkl'

In [121]:
summary = get_statistical_indicators(path)

F:\anaconda\envs\ricequant\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  from ipykernel import kernelapp as app
F:\anaconda\envs\ricequant\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [122]:
summary

{'起始日': '2014-03-31',
 '标的资产': '000300.XSHG',
 '股票数': 173,
 'Beta': 0.974,
 '跟踪误差': 0.035,
 '年化收益率': 0.215,
 '基准年化收益率': 0.139,
 '十大持仓股':               symbol    weight
 order_book_id                 
 600519.XSHG     贵州茅台  0.075496
 000858.XSHE      五粮液  0.042134
 000651.XSHE     格力电器  0.039866
 002352.XSHE     顺丰控股  0.030723
 600276.XSHG     恒瑞医药  0.026996
 600383.XSHG     金地集团  0.026196
 000656.XSHE     金科股份  0.025437
 300498.XSHE     温氏股份  0.023241
 002415.XSHE     海康威视  0.022550
 600031.XSHG     三一重工  0.022140,
 '行业配置':             weight
 industry          
 食品饮料      0.147336
 医药生物      0.141226
 电子        0.079349
 房地产       0.077201
 电气设备      0.070119
 农林牧渔      0.060516
 家用电器      0.052565
 机械设备      0.046517
 化工        0.044423
 传媒        0.039283
 汽车        0.035309
 计算机       0.034466
 交通运输      0.030723
 建筑材料      0.025698
 通信        0.024845
 轻工制造      0.024028
 休闲服务      0.016499
 综合        0.011001
 非银金融      0.010182
 公用事业      0.006922
 国防军工      0.006884
 有色金属      

In [124]:
summary['净值'].to_csv('净值.csv')

In [125]:
summary['组合与基准收益']

,1年,3年,5年,10年,至今
组合资产,0.407617,0.629183,0.759703,NaN,2.357397
标的资产,0.280856,0.301458,0.211415,NaN,1.256849
